# 0. 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display as dp
import warnings
import matplotlib
import matplotlib.pyplot as plt

# 스케일링 => KNN은 거리 기반이므로 이상치에 민감하니 StandardScaler 사용
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from time import time

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 검증과 테스트 데이터셋은 피처 스케일링되어 사용되므로 먼저 정의하였다.
validation =pd.read_csv('../data/validation_encoding.csv', index_col=0)
test =pd.read_csv('../data/test_encoding.csv', index_col=0)

X_val, y_val = validation.iloc[:,:-1], validation.iloc[:,-1] # 그대로 사용됨.
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1] # 그대로 사용됨.

dp(validation,test)
dp(X_val, y_val.to_frame() , X_test, y_test.to_frame())

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,2
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,1.0,0.0,0.0,0.0,0.0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,1.0,0.0,0.0,0.0,0.0,1,1,0,1,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,1.0,0.0,1,0,0,0,2
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,0,1
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,1.0,0.0,0,0,1,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,1


,TARGET
73378,2
71524,1
38781,0
35212,0
56852,1
...,...
23050,0
36543,0
70132,0
58097,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,0
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,1,0
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


,TARGET
12635,1
22195,1
42119,2
34954,0
65122,0
...,...
41734,1
16135,1
65530,1
80594,0


# 1 베이스 모델 강화하기
파라미터 공간을 크게 늘려서 돌려보기

In [2]:
# 피처 스케일링에 사용될 scaler
scaler = MinMaxScaler()

# base model의 학습 데이터
base_train = pd.read_csv('../data/base_train_encoding.csv', index_col=0)
X_train_base, y_train_base = base_train.iloc[:,:-1], base_train.iloc[:,-1]

X_train_base_scaled = scaler.fit_transform(X_train_base)

# 어떤 모델이건 사용될 검증/테스트 데이터셋 # 이게 아니라는 것. 훈련데이터셋이 바뀔 때마다 새롭게 transform 해야함. 이건 그냥 base_model의 경우임.
X_val_base_scaled = scaler.transform(X_val)
X_test_base_scaled = scaler.transform(X_test)

X_train_base_scaled.shape # pd.DataFrame(X_train_base_scaled, index = X_train_base.index, columns = X_train_base.columns)

(50643, 21)

In [ ]:
from time import time
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(10,), (20,), (10,2), (20,2), (10,5,2),(20,10,2)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'max_iter': [1000, 2000, 3000]
}
grid_search = GridSearchCV(mlp, params, cv = 5, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')

In [4]:
params_base = grid_search.best_params_
print(params_base)

{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'max_iter': 2000, 'solver': 'adam'}


# 2. 학습

In [5]:
# mlp 모델 학습
start = time()

mlp_base = MLPClassifier(hidden_layer_sizes = params_base['hidden_layer_sizes'],
                         alpha = params_base['alpha'],
                         solver = params_base['solver'],
                         activation = params_base['activation'],
                         learning_rate = params_base['learning_rate'],
                         max_iter = params_base['max_iter'])
mlp_base.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 16.238713026046753
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.57      0.36      0.44      5052
           2       0.44      0.40      0.42      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.53      0.53      0.52     16881

[[5292  764 1220]
 [2094 1830 1128]
 [2086  625 1842]]
0.5310111960191932
0.5310111960191932


# 3. 모델 저장

In [6]:
# from joblib import dump, load
# dump(mlp_base, '../model/MLP_base_MinMax.joblib')

# # # 모델 불러오기
# # model = load('../model/MLP_base_MinMax.joblib')

# 4. 파라미터 찾아보기
{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'max_iter': 2000, 'solver': 'adam'}가 최선으로 나왔으나 정확도가 약 53%로 별 다를 것이 없다.

In [9]:
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(20,), (30,),(40,)], 
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.05],
    'learning_rate': ['adaptive'],
    'max_iter': [2000, 3000]
}
grid_search2 = GridSearchCV(mlp, params, cv = 3, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
params_base2 = grid_search.best_params_
print(params_base2)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   6.6s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   6.5s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   7.9s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=3000, solver=adam; total time=   7.0s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=3000, solver=adam; total time=   6.9s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=3000, solver=adam; total time=  10.0s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(30,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=  1

In [10]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 11.78928828239441
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.54      0.39      0.45      5052
           2       0.45      0.37      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.52      0.53      0.52     16881

[[5306  875 1095]
 [2096 1951 1005]
 [2102  755 1696]]
0.530359575854511
0.530359575854511


In [11]:
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(20,),(10,1),(20,1)], 
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.05],
    'learning_rate': ['adaptive'],
    'max_iter': [2000]
}
grid_search2 = GridSearchCV(mlp, params, cv = 3, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
params_base2 = grid_search.best_params_
print(params_base2)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   5.7s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   7.8s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   7.4s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(10, 1), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   1.2s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(10, 1), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   9.1s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(10, 1), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   1.3s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20, 1), learning_rate=adaptive, max_iter=2000, solver=adam; total t

In [12]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 7.474471569061279
              precision    recall  f1-score   support

           0       0.56      0.72      0.63      7276
           1       0.56      0.39      0.46      5052
           2       0.45      0.39      0.42      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.53      0.53      0.52     16881

[[5261  862 1153]
 [2042 1960 1050]
 [2072  702 1779]]
0.5331437711036076
0.5331437711036076


In [ ]:
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(20,),(19,),(18,)], 
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.05],
    'learning_rate': ['adaptive'],
    'max_iter': [1000, 2000]
}
grid_search2 = GridSearchCV(mlp, params, cv = 3, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')

In [4]:
params_base2 = grid_search2.best_params_
print(params_base2)

{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'max_iter': 2000, 'solver': 'adam'}


In [6]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base2['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 6.7495810985565186
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.57      0.36      0.44      5052
           2       0.44      0.40      0.42      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.53      0.53      0.52     16881

[[5326  771 1179]
 [2110 1832 1110]
 [2105  633 1815]]
0.5315443397902968
0.5315443397902968


In [7]:
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(20,),(20,1)], 
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.05],
    'learning_rate': ['adaptive'],
    'max_iter': [2000]
}
grid_search2 = GridSearchCV(mlp, params, cv = 3, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=sgd; total time=   9.8s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=sgd; total time=  13.6s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=sgd; total time=  17.5s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=  18.6s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=  16.8s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   9.5s
[CV] END activation=tanh, alpha=0.05, hidden_layer_sizes=(20, 1), learning_rate=adaptive, max_iter=2000, solver=sgd; total time=  13.

In [9]:
params_base2 = grid_search2.best_params_
print(params_base2)

{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'max_iter': 2000, 'solver': 'adam'}


In [8]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base2['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 8.689563512802124
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.56      0.38      0.45      5052
           2       0.44      0.39      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.53      0.53      0.52     16881

[[5283  832 1161]
 [2078 1911 1063]
 [2092  691 1770]]
0.5310111960191932
0.5310111960191932


In [12]:
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV

params = {
    'hidden_layer_sizes': [(20,),(21,),(22,)], 
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.05],
    'learning_rate': ['adaptive'],
    'max_iter': [2000]
}
grid_search2 = GridSearchCV(mlp, params, cv = 3, verbose = 2) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
grid_search2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   4.7s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   4.4s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(20,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   6.4s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(21,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   5.5s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(21,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   7.1s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(21,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   3.3s
[CV] END activation=relu, alpha=0.05, hidden_layer_sizes=(22,), learning_rate=adaptive, max_iter=2000, solver=adam; total time=   6

In [14]:
params_base2 = grid_search2.best_params_
print(params_base2)

{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (21,), 'learning_rate': 'adaptive', 'max_iter': 2000, 'solver': 'adam'}


In [13]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base2['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 23.310771942138672
              precision    recall  f1-score   support

           0       0.55      0.73      0.63      7276
           1       0.55      0.35      0.43      5052
           2       0.43      0.39      0.41      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.49      0.49     16881
weighted avg       0.52      0.52      0.51     16881

[[5302  790 1184]
 [2179 1758 1115]
 [2124  668 1761]]
0.5225401338783248
0.5225401338783248


In [4]:
from skopt import BayesSearchCV

In [3]:
from sklearn.model_selection import RandomizedSearchCV
from time import time
start = time()

mlp = MLPClassifier()

# 적절한 hidden_layer_size와 max_iter값을 위한 과정

params = {
    'hidden_layer_sizes': [(20,1), (20,), (21,),], 
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': (0.00001, 0.1, 'log-uniform'),  
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': (0.0001, 0.1, 'log-uniform'),
    'momentum': (0.1, 0.9, 'uniform'),
    'nesterovs_momentum': [True, False],
    'early_stopping': [True, False],
    'validation_fraction': (0.1, 0.2, 'uniform'),
    'max_iter': [1000, 2000, 3000, 4000]
}
random_search = RandomizedSearchCV(mlp, params, cv = 3, verbose = 2, n_iter=50) 
random_search.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
params_base2 = random_search.best_params_
print(params_base2)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END activation=logistic, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.0001, max_iter=2000, momentum=uniform, nesterovs_momentum=False, solver=adam, validation_fraction=0.2; total time=   0.0s
[CV] END activation=logistic, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.0001, max_iter=2000, momentum=uniform, nesterovs_momentum=False, solver=adam, validation_fraction=0.2; total time=   0.0s
[CV] END activation=logistic, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.0001, max_iter=2000, momentum=uniform, nesterovs_momentum=False, solver=adam, validation_fraction=0.2; total time=   0.0s
[CV] END activation=tanh, alpha=log-uniform, early_stopping=False, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.0001, max_i

[CV] END activation=tanh, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.1, max_iter=1000, momentum=0.1, nesterovs_momentum=True, solver=sgd, validation_fraction=0.1; total time=  17.9s
[CV] END activation=tanh, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.1, max_iter=1000, momentum=0.1, nesterovs_momentum=True, solver=sgd, validation_fraction=0.1; total time=  16.6s
[CV] END activation=tanh, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(20,), learning_rate=adaptive, learning_rate_init=0.1, max_iter=1000, momentum=0.1, nesterovs_momentum=True, solver=sgd, validation_fraction=0.1; total time=  15.9s
[CV] END activation=relu, alpha=0.1, early_stopping=False, hidden_layer_sizes=(21,), learning_rate=adaptive, learning_rate_init=0.0001, max_iter=3000, momentum=0.9, nesterovs_momentum=False, solver=adam, validation_fraction=0.2; total time=  10.9s
[CV] END activati

[CV] END activation=logistic, alpha=0.1, early_stopping=False, hidden_layer_sizes=(21,), learning_rate=adaptive, learning_rate_init=log-uniform, max_iter=4000, momentum=0.1, nesterovs_momentum=False, solver=sgd, validation_fraction=uniform; total time=   0.0s
[CV] END activation=relu, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=4000, momentum=0.9, nesterovs_momentum=True, solver=adam, validation_fraction=0.2; total time=   0.0s
[CV] END activation=relu, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=4000, momentum=0.9, nesterovs_momentum=True, solver=adam, validation_fraction=0.2; total time=   0.0s
[CV] END activation=relu, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=4000, momentum=0.9, nesterovs_momentum=True, solver=adam, validation_fraction=0.

[CV] END activation=relu, alpha=log-uniform, early_stopping=True, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=log-uniform, max_iter=1000, momentum=0.1, nesterovs_momentum=False, solver=adam, validation_fraction=uniform; total time=   0.0s
[CV] END activation=relu, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=1000, momentum=0.1, nesterovs_momentum=False, solver=adam, validation_fraction=uniform; total time=   0.0s
[CV] END activation=relu, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=1000, momentum=0.1, nesterovs_momentum=False, solver=adam, validation_fraction=uniform; total time=   0.0s
[CV] END activation=relu, alpha=1e-05, early_stopping=False, hidden_layer_sizes=(21,), learning_rate=constant, learning_rate_init=0.0001, max_iter=1000, momentum=0.1, nesterovs_momentum=False, solver=adam, validation_fraction=un

In [4]:
# mlp 모델 학습
start = time()

mlp_base2 = MLPClassifier(hidden_layer_sizes = params_base2['hidden_layer_sizes'],
                         alpha = params_base2['alpha'],
                         solver = params_base2['solver'],
                         activation = params_base2['activation'],
                         learning_rate = params_base2['learning_rate'],
                         max_iter = params_base2['max_iter'])
mlp_base2.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_base2.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 34.12999176979065
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.55      0.36      0.44      5052
           2       0.43      0.39      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.51      0.49      0.49     16881
weighted avg       0.52      0.53      0.51     16881

[[5301  777 1198]
 [2115 1814 1123]
 [2085  690 1778]]
0.5268052840471537
0.5268052840471537
